<a href="https://colab.research.google.com/github/Jdka1/Kaggle-Disaster-Tweet-Prediction/blob/main/Kaggle_Tweet_Analysis_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"aryanmehra","key":"02cedc58913d4420bbafb80ff315eed5"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c nlp-getting-started
!unzip nlp-getting-started.zip

nlp-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  nlp-getting-started.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [ ]:
import pandas as pd
import numpy as np
from sklearn import feature_extraction, linear_model, model_selection, preprocessing


train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
count_vectorizer = feature_extraction.text.CountVectorizer()

train_counts = count_vectorizer.fit_transform(train_df['text'])
print(f'Number of unique words: {train_counts[0].todense().shape[1]}')

test_counts = count_vectorizer.transform(test_df['text'])

Number of unique words: 21637


In [ ]:
# Using ridge regression (to push the coeffiecients towards 0)
# This allows the model to avoid overfitting
# Therefore it will work better on new data
clf = linear_model.RidgeClassifier()
clf.fit(train_counts, train_df['target'])

RidgeClassifier()

In [ ]:
submission = pd.read_csv('test.csv')
submission['target'] = clf.predict(test_counts)
submission

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_counts, train_df['target'], test_size=0.33)

y_pred = clf.predict(X_test)
y_actual = list(y_test)

# for i in range(len(y_pred)):
#   print(f'{y_pred[i]} : {y_actual[i]}')

def recall(pred, actual):
  # Check how many true positives were missed
  recall_df = pd.DataFrame(columns=['Pred','Actual'])
  recall_df['Pred'], recall_df['Actual'] = pred, actual
  recall_df = recall_df.where(recall_df['Actual'] == 1).dropna()

  num_total = len(recall_df.index)
  num_positve = len(recall_df.where(recall_df['Pred'] == recall_df['Actual']).dropna().index)

  recall = num_positve / num_total
  return recall

def precision(pred, actual):
  # Check how many positives were true positives
  accuracy_df = pd.DataFrame(columns=['Pred','Actual'])
  accuracy_df['Pred'], accuracy_df['Actual'] = pred, actual
  accuracy_df = accuracy_df.where(accuracy_df['Pred'] == 1).dropna()

  num_positives = len(accuracy_df.index)
  num_true_positives = len(accuracy_df.where(accuracy_df['Actual'] == 1).dropna().index)

  return num_true_positives / num_positives


print(f'Precision: {round(precision(y_pred, y_actual) * 100, 3)}%')
print(f'Recall: {round(recall(y_pred, y_actual) * 100, 3)}%')

Precision: 99.714%
Recall: 99.335%
